In [1]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
print('This notebook was run on ' + time.strftime("%d/%m/%Y") + ' at ' + time.strftime("%H:%M:%S") + '.')

This notebook was run on 26/09/2023 at 12:11:37.


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [4]:
dataset_directory = os.path.join(os.getenv('HOME'), 'icar-ng-data', 'dataset')
print('Dataset directory: {}'.format(dataset_directory))

dataset_file = 'dataset.csv'
print('Dataset file: {}'.format(dataset_file))

dataset_path = os.path.join(dataset_directory, dataset_file)
print('Dataset path: {}'.format(dataset_path))

# ======================================

model_directory = os.path.join(os.getenv('HOME'), 'icar-ng-data', 'model')
print('Model directory: {}'.format(model_directory))

model_file = 'pixel2cm_model.pt'
print('Model file: {}'.format(model_file))

model_path = os.path.join(model_directory, model_file)
print('Model path: {}'.format(model_path))

Dataset directory: /home/protanjung/icar-ng-data/dataset
Dataset file: dataset.csv
Dataset path: /home/protanjung/icar-ng-data/dataset/dataset.csv
Model directory: /home/protanjung/icar-ng-data/model
Model file: pixel2cm_model.pt
Model path: /home/protanjung/icar-ng-data/model/pixel2cm_model.pt


In [5]:
# Check whether dataset file exists
if not os.path.exists(dataset_path):
    print('Dataset file does not exist: {}'.format(dataset_path))
    exit()

# Check whether model directory exists
if not os.path.exists(model_directory):
    os.makedirs(model_directory)
    print('Created model directory: {}'.format(model_directory))

In [6]:
df_raw = pd.read_csv(dataset_path)
df_train = df_raw.sample(frac=0.8)
df_test = df_raw.drop(df_train.index)

min_x = np.min(df_train.iloc[:, 0:2].values, axis=0)
min_x = torch.tensor(min_x, dtype=torch.float32)
max_x = np.max(df_train.iloc[:, 0:2].values, axis=0)
max_x = torch.tensor(max_x, dtype=torch.float32)
min_y = np.min(df_train.iloc[:, 2:4].values, axis=0)
min_y = torch.tensor(min_y, dtype=torch.float32)
max_y = np.max(df_train.iloc[:, 2:4].values, axis=0)
max_y = torch.tensor(max_y, dtype=torch.float32)

print('min_x: {}, max_x: {}'.format(min_x, max_x))
print('min_y: {}, max_y: {}'.format(min_y, max_y))

min_x: tensor([ 30., 260.]), max_x: tensor([1268.,  692.])
min_y: tensor([   0., -300.]), max_y: tensor([450., 300.])


In [7]:
class ForwardDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
        x = self.dataframe.iloc[:, 0:2].values
        y = self.dataframe.iloc[:, 2:4].values
        self.x = torch.from_numpy(x).float()
        self.y = torch.from_numpy(y).float()
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [8]:
dataset_train = ForwardDataset(df_train)
dataset_test = ForwardDataset(df_test)
print('dataset_train: {}'.format(len(dataset_train)))
print('dataset_test: {}'.format(len(dataset_test)))

dataloader_train = DataLoader(dataset_train, batch_size=64, shuffle=True, pin_memory=True)
dataloader_test = DataLoader(dataset_test, batch_size=64, shuffle=True, pin_memory=True)

dataset_train: 91
dataset_test: 23


In [9]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_size, output_size, min_x, max_x, min_y, max_y):
        super(MultiLayerPerceptron, self).__init__()
        self.min_x = min_x
        self.max_x = max_x
        self.min_y = min_y
        self.max_y = max_y
        self.fc1 = nn.Linear(input_size, 4)
        self.fc2 = nn.Linear(4, 16)
        self.fc3 = nn.Linear(16, 64)
        self.fc4 = nn.Linear(64, 16)
        self.fc5 = nn.Linear(16, 4)
        self.fc6 = nn.Linear(4, output_size)

    def forward(self, x):
        x = (x - self.min_x) / (self.max_x - self.min_x)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.fc6(x)
        x = x * (self.max_y - self.min_y) + self.min_y
        return x

In [10]:
model = MultiLayerPerceptron(2, 2, min_x.to(device), max_x.to(device), min_y.to(device), max_y.to(device)).to(device)

if os.path.exists(model_path):
    try:
        print('Loading model: {}'.format(model_path))
        model.load_state_dict(torch.load(model_path))
        print('Loaded model: {}'.format(model_path))
    except BaseException as e:
        print('Failed to load model: {}'.format(model_path))
        print(e)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

Loading model: /home/protanjung/icar-ng-data/model/pixel2cm_model.pt
Loaded model: /home/protanjung/icar-ng-data/model/pixel2cm_model.pt


In [11]:
min_test_loss = np.inf

for epoch in range(50000):
    model.train()
    train_loss = 0.0
    for i, (x, y) in enumerate(dataloader_train):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    if epoch % 100 == 0:
        model.eval()
        test_loss = 0.0
        for i, (x, y) in enumerate(dataloader_test):
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            test_loss += loss.item()

        save_model = False
        if test_loss < min_test_loss:
            min_test_loss = test_loss
            torch.save(model.state_dict(), model_path)
            save_model = True

        if save_model:
            print('epoch: {}, train_loss: {:.6f}, test_loss: {:.6f} (Saved)'.format(epoch, train_loss, test_loss))
        else:
            print('epoch: {}, train_loss: {:.6f}, test_loss: {:.6f}'.format(epoch, train_loss, test_loss))

epoch: 0, train_loss: 82969.531250, test_loss: 31843.570312 (Saved)
epoch: 100, train_loss: 70820.970703, test_loss: 27318.500000 (Saved)
epoch: 200, train_loss: 62231.236328, test_loss: 24863.410156 (Saved)
epoch: 300, train_loss: 62416.578125, test_loss: 22909.375000 (Saved)
epoch: 400, train_loss: 53197.500000, test_loss: 21184.654297 (Saved)
epoch: 500, train_loss: 57136.605469, test_loss: 19707.726562 (Saved)
epoch: 600, train_loss: 49580.169922, test_loss: 18230.568359 (Saved)
epoch: 700, train_loss: 49283.458984, test_loss: 17022.142578 (Saved)
epoch: 800, train_loss: 45944.542969, test_loss: 15972.717773 (Saved)
epoch: 900, train_loss: 45616.275391, test_loss: 14928.288086 (Saved)
epoch: 1000, train_loss: 40483.275391, test_loss: 13960.291016 (Saved)
epoch: 1100, train_loss: 35591.335938, test_loss: 13134.852539 (Saved)
epoch: 1200, train_loss: 36202.275391, test_loss: 12441.505859 (Saved)
epoch: 1300, train_loss: 31636.343750, test_loss: 11675.589844 (Saved)
epoch: 1400, train

In [12]:
onnx_path = os.path.join(model_directory, 'pixel2cm_model.onnx')
print('ONNX path: {}'.format(onnx_path))

try:
    print('Saving ONNX model: {}'.format(onnx_path))
    torch.onnx.export(model, torch.randn(1, 2).to(device), onnx_path, verbose=True)
    print('Saved ONNX model: {}'.format(onnx_path))
except BaseException as e:
    print('Failed to save ONNX model: {}'.format(onnx_path))
    print(e)

ONNX path: /home/protanjung/icar-ng-data/model/pixel2cm_model.onnx
Saving ONNX model: /home/protanjung/icar-ng-data/model/pixel2cm_model.onnx
Exported graph: graph(%onnx::Sub_0 : Float(1, 2, strides=[2, 1], requires_grad=0, device=cuda:0),
      %fc1.weight : Float(4, 2, strides=[2, 1], requires_grad=1, device=cuda:0),
      %fc1.bias : Float(4, strides=[1], requires_grad=1, device=cuda:0),
      %fc2.weight : Float(16, 4, strides=[4, 1], requires_grad=1, device=cuda:0),
      %fc2.bias : Float(16, strides=[1], requires_grad=1, device=cuda:0),
      %fc3.weight : Float(64, 16, strides=[16, 1], requires_grad=1, device=cuda:0),
      %fc3.bias : Float(64, strides=[1], requires_grad=1, device=cuda:0),
      %fc4.weight : Float(16, 64, strides=[64, 1], requires_grad=1, device=cuda:0),
      %fc4.bias : Float(16, strides=[1], requires_grad=1, device=cuda:0),
      %fc5.weight : Float(4, 16, strides=[16, 1], requires_grad=1, device=cuda:0),
      %fc5.bias : Float(4, strides=[1], requires_gr

In [13]:
model.eval()
for i, (x, y) in enumerate(dataloader_test):
    x = x.to(device)
    y = y.to(device)
    y_pred = model(x)
    
    print('x\n{}'.format(x))
    print('y\n{}'.format(y))
    print('y_pred\n{}'.format(y_pred))
    print('y - y_pred\n{}'.format(y - y_pred))
    
    break
     

x
tensor([[ 570.,  290.],
        [1212.,  678.],
        [ 716.,  442.],
        [ 524.,  268.],
        [ 520.,  394.],
        [ 640.,  598.],
        [ 938.,  524.],
        [ 122.,  452.],
        [ 838.,  528.],
        [ 488.,  446.],
        [1000.,  324.],
        [ 204.,  336.],
        [ 948.,  438.],
        [ 264.,  452.],
        [ 746.,  352.],
        [ 822.,  324.],
        [1272.,  342.],
        [ 158.,  532.],
        [ 874.,  594.],
        [ 910.,  324.],
        [1024.,  432.],
        [ 954.,  348.],
        [ 416.,  446.]], device='cuda:0')
y
tensor([[ 350.,  -50.],
        [   0.,  100.],
        [ 100.,   25.],
        [ 450., -100.],
        [ 150.,  -50.],
        [  25.,    0.],
        [  50.,   75.],
        [ 100., -175.],
        [  50.,   50.],
        [ 100.,  -50.],
        [ 250.,  200.],
        [ 250., -250.],
        [ 100.,  100.],
        [ 100., -125.],
        [ 200.,   50.],
        [ 250.,  100.],
        [ 200.,  300.],
        [  50., -1